In [3]:
import salabim as sim

env = sim.Environment(time_unit='seconds')

customer_to_log = 500

#Shopping baskets and carts:
carts = env.Resource('carts', capacity=45) #45
baskets = env.Resource('baskets', capacity=500)#several hundred
cart_basket_distribution = sim.Pdf((carts, 0.8,baskets, 0.2))
customer_basketcart_distribution_monitor = sim.Monitor("Customer basket and cart monitor")
#shopping categories:
# fruit_and_vegetables 
# meat_and_fish
# bread
# cheese_and_dairy
# canned_and_packed_food
# frozen_foods
# drinks
item_taking_distribution = sim.Uniform(20,30)

#Clerks
#Bread and cheese use resources as they are working via counters. Checkout likely has to use queue as the customers need to choose the smallest one
bread_clerks = env.Resource('bread_clerks', capacity=4) #4 employees, 1-6 items takes 2 min
bread_time_distribution = sim.Exponential(2*60)
cheese_and_dairy_clerks = env.Resource('bread_clerks', capacity=3) #3 employees, 1 min avg.
cheese_and_dairy_time_distribution = sim.Exponential(1*60)

#Checkouts
number_of_checkouts = 3
time_per_item_distribution = sim.Exponential(1.1)
payment_time_distribution = sim.Uniform(40, 60)

#Distributions of items per customer
fruit_and_vegetables_distribution = sim.Triangular(4, 22,10)  # min=4, mode=10, max=22
meat_and_fish_distribution = sim.Triangular(0, 9, 4)           # min=0, mode=4, max=9
bread_distribution = sim.Triangular(1, 10, 4)                  # min=1, mode=4, max=10
cheese_and_dairy_distribution = sim.Triangular(1, 11, 3)       # min=1, mode=3, max=11
canned_and_packed_food_distribution = sim.Triangular(6, 35, 17)# min=6, mode=17, max=35
frozen_foods_distribution = sim.Triangular(2, 19, 8)           # min=2, mode=8, max=19
drinks_distribution = sim.Triangular(1, 20, 9)                 # min=1, mode=9, max=20

#Route choice distribution

#Routes
route1 = [
"fruit_and_vegetables",
"meat_and_fish",
"bread",
"cheese_and_dairy",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #ABCDEF, 80% 

route2 = [
"meat_and_fish",
"bread",
"cheese_and_dairy",
"fruit_and_vegetables",
"canned_and_packed_food",
"frozen_foods",
"drinks",
] #BCDEFG, 20%

route_distribution = sim.Pdf((route1, 0.8, route2, 0.2))

class Customer(sim.Component):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.route = route_distribution.sample()
        self.shopping_list = {
            "fruit_and_vegetables": int(round(fruit_and_vegetables_distribution.sample())),
            "meat_and_fish": int(round(meat_and_fish_distribution.sample())),
            "bread": int(round(bread_distribution.sample())),
            "cheese_and_dairy": int(round(cheese_and_dairy_distribution.sample())),
            "canned_and_packed_food": int(round(canned_and_packed_food_distribution.sample())),
            "frozen_foods": int(round(frozen_foods_distribution.sample())),
            "drinks": int(round(drinks_distribution.sample())),
        }
        self.progress = 0
        self.carrying = None
        self.actions_log = []

    def log_action(self, action):
            """Helper function to log an action with the current time."""
            self.actions_log.append((env.now(), action))
        
    def process(self):
        """"
        Process determines what the customer will do. At the start they will take a cart or basket. Afterwards they will traverse their route and take the items they need according to their shopping list. If they have finished their route (when progress is equal to the length of the shopping list), they will go to the checkout.
        """
        self.start_shopping()
        for next_product in self.route:
            if self.shopping_list[next_product]>0:
                self.get_product(next_product)   
        self.go_to_checkout()
            
    def start_shopping(self):
        #Get shopping cart or basket
        want_to_carry = cart_basket_distribution.sample()
        self.log_action(f"Entered cart/basket queue for {want_to_carry}")
        self.request(want_to_carry)
        customer_basketcart_distribution_monitor.tally(want_to_carry)
        self.log_action(f"Got {want_to_carry}")
        self.carrying = want_to_carry
        
    def go_to_checkout(self):
        #enter emptiest queue
        emptiest_queue = min(checkouts, key=lambda checkout: checkout.requesters().length()) 
        self.log_action(f"Entered checkout queue {emptiest_queue}")
        self.request(emptiest_queue) 
        self.log_action(f"Started checking out")
        item_scan_time = sum(time_per_item_distribution.sample() for _ in range(sum(self.shopping_list.values())))
        self.hold(item_scan_time+payment_time_distribution.sample()) #hold the customer for scanning all items and during payment
        self.log_action(f"Finished checking out")
        #return cart/basket       
        #print log if we want to debug
        if customer_to_log:
            if self.name() == f"customer.{customer_to_log}":
                print(self.carrying.claimers().print_info())
                print(f"Customer's Action Log for customer {self.name()}:")
                for time, action in self.actions_log:
                    print(f"At time {time}, customer: {action}")
        
        
    def get_product(self, product):
        #Move to product location #<- only needed in animation
        if product == "cheese_and_dairy":
            self.log_action(f"requesting cheese and dairy")
            self.request(cheese_and_dairy_clerks)
            self.log_action(f"Being helped for cheese and dairy")
            self.hold(cheese_and_dairy_distribution.sample())
            self.log_action(f"Got cheese and dairy")
            self.release(cheese_and_dairy_clerks)
        elif product == "bread":
            self.log_action(f"requesting bread")
            self.request(bread_clerks)
            self.log_action(f"Being helped for bread")
            self.hold(bread_time_distribution.sample())
            self.log_action(f"Got bread")
            self.release(bread_clerks)
        else:
            amount = self.shopping_list[product]
            self.log_action(f"Getting {product}")
            for _ in range(amount):
                self.hold(time_per_item_distribution.sample())
                self.log_action(f"Got {product}")

#customer generation
customer_distribution = [30, 80, 110, 90, 80, 70, 80, 90, 100, 120, 90, 40] #Expected total = 980
for index, customer_count  in enumerate(customer_distribution):
    env.ComponentGenerator(Customer, iat=env.Exponential(3600/customer_count), at=index*60*60, duration=60*60) #assumes time in seconds
customer_arrival_monitor = sim.Monitor("Customer arrival monitor")

checkouts = [] #dictionary to map checkouts to clerks
for i in range(number_of_checkouts):
    checkouts.append(env.Resource(f"checkout_clerk{i}", capacity = 1)) #3, 1.1s per item avg. payment 40-60s

env.run(duration=60*60*12)


#Statistics:
# for checkout in checkouts:
#     checkout.length.print_histograms()
#carts.print_statistics()
# carts.print_info()
bread_clerks.print_statistics()
bread_clerks.print_info()
cheese_and_dairy_clerks.print_statistics()
# cheese_and_dairy_clerks.print_info()
#customer_basketcart_distribution_monitor.print_histogram(values=True)


Queue 0x1f9bfa9bd90
  name=claimers of carts
  component(s):
    customer.500         enter_time 22160.299 priority=inf
    customer.503         enter_time 22338.055 priority=inf
    customer.507         enter_time 22425.834 priority=inf
    customer.509         enter_time 22436.402 priority=inf
    customer.512         enter_time 22556.216 priority=inf
    customer.514         enter_time 22739.755 priority=inf
    customer.515         enter_time 22762.694 priority=inf
    customer.516         enter_time 22896.058 priority=inf
    customer.517         enter_time 22961.416 priority=inf
    customer.518         enter_time 23033.253 priority=inf
    customer.519         enter_time 23090.725 priority=inf
    customer.520         enter_time 23103.833 priority=inf
    customer.521         enter_time 23188.734 priority=inf
    customer.522         enter_time 23249.490 priority=inf
    customer.523         enter_time 23252.945 priority=inf
    customer.524         enter_time 23253.231 priority

{'fruit_and_vegetables': 12, 'meat_and_fish': 3, 'bread': 6, 'cheese_and_dairy': 3, 'canned_and_packed_food': 17, 'frozen_foods': 16, 'drinks': 10}


In [12]:
help(sim.Exponential)

Help on class Exponential in module salabim.salabim:

class Exponential(_Distribution)
 |  Exponential(mean: float = None, time_unit: str = None, rate: float = None, randomstream: Any = None, env: 'Environment' = None)
 |  
 |  exponential distribution
 |  
 |  Parameters
 |  ----------
 |  mean : float
 |      mean of the distribtion (beta)
 |  
 |      if omitted, the rate is used
 |  
 |      must be >0
 |  
 |  time_unit : str
 |      specifies the time unit
 |  
 |      must be one of "years", "weeks", "days", "hours", "minutes", "seconds", "milliseconds", "microseconds"
 |  
 |      default : no conversion
 |  
 |  
 |  rate : float
 |      rate of the distribution (lambda)
 |  
 |      if omitted, the mean is used
 |  
 |      must be >0
 |  
 |  randomstream: randomstream
 |      randomstream to be used
 |  
 |      if omitted, random will be used
 |  
 |      if used as random.Random(12299)
 |      it assigns a new stream with the specified seed
 |  
 |  env : Environment
 |  

In [8]:
help(sim.Triangular(4, 22,10))

Help on Triangular in module salabim.salabim object:

class Triangular(_Distribution)
 |  Triangular(low: float, high: float = None, mode: float = None, time_unit: str = None, randomstream: Any = None, env: 'Environment' = None)
 |  
 |  triangular distribution
 |  
 |  Parameters
 |  ----------
 |  low : float
 |      lowerbound of the distribution
 |  
 |  high : float
 |      upperbound of the distribution
 |  
 |      if omitted, low will be used, thus effectively a constant distribution
 |  
 |      high must be >= low
 |  
 |  mode : float
 |      mode of the distribution
 |  
 |      if omitted, the average of low and high will be used, thus a symmetric triangular distribution
 |  
 |      mode must be between low and high
 |  
 |  time_unit : str
 |      specifies the time unit
 |  
 |      must be one of "years", "weeks", "days", "hours", "minutes", "seconds", "milliseconds", "microseconds"
 |  
 |      default : no conversion
 |  
 |  
 |  randomstream: randomstream
 |      r

In [2]:
import salabim as sim
help(sim.Pdf)

Help on class Pdf in module salabim.salabim:

class Pdf(_Distribution)
 |  Pdf(spec: Union[Iterable, Dict], probabilities=None, time_unit: str = None, randomstream: Any = None, env: 'Environment' = None)
 |  
 |  Probability distribution function
 |  
 |  Parameters
 |  ----------
 |  spec : list, tuple or dict
 |      either
 |  
 |      -   if no probabilities specified:
 |  
 |          list/tuple with x-values and corresponding probability
 |          dict where the keys are re x-values and the values are probabilities
 |          (x0, p0, x1, p1, ...xn,pn)
 |  
 |      -   if probabilities is specified:
 |  
 |          list with x-values
 |  
 |  probabilities : iterable or float
 |      if omitted, spec contains the probabilities
 |  
 |      the iterable (p0, p1, ...pn) contains the probabilities of the corresponding
 |      x-values from spec.
 |  
 |      alternatively, if a float is given (e.g. 1), all x-values
 |      have equal probability. The value is not important.
 |  